In [1]:
!pip install pandas
!pip install plotly


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 23.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [48]:
import pandas as pd
file_path = "../datasets/datascience_salaries_2023.csv"
salaries_df = pd.read_csv(file_path)
#info gives us the column name, nn count, and data type
salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


In [49]:
#filter the data to only use US FT positions in 2023
salaries_2023_df = salaries_df[salaries_df['work_year'] == 2023]
salaries_2023_US_df = salaries_2023_df[salaries_2023_df['company_location'] == 'US']
salaries_2023_US_FT_df = salaries_2023_US_df[salaries_2023_US_df['employment_type'] == 'FT']

In [57]:
salaries_df2 = salaries_2023_US_FT_df

def get_salary_bins(salaries_df, number_of_bins=5):
    salary_column = salaries_df['salary_in_usd']
    min_salary = salary_column.min()
    max_salary = salary_column.max()
    bin_width = (max_salary - min_salary) / number_of_bins
    bins = [min_salary + i * bin_width for i in range(number_of_bins + 1)]
    return bins

bins = get_salary_bins(salaries_df2)
# Group the salaries into bins.  have to use .loc to get around error "A value is trying to be set on a copy of a slice from a DataFrame"
salaries_df2.loc[:, 'Salary Bin'] = pd.cut(salaries_df2['salary_in_usd'], bins)
print(salaries_df2)
# Categories (5, interval[float64, right]): [(5132.0, 94105.6] < (94105.6, 183079.2] < (183079.2, 272052.8] < (272052.8, 361026.4] < (361026.4, 450000.0]]

# Count the number of occurrences in each bin and sort from lowest to highest category. i.e. list (5132.0 - 94105.6), (94105.6 - 183079.2), ...
bin_counts = salaries_df2['Salary Bin'].value_counts().sort_index()
print(bin_counts)
# (30000.0, 101000.0]     237
# (101000.0, 172000.0]    793
# (172000.0, 243000.0]    402
# (243000.0, 314000.0]    120
# (314000.0, 385000.0]     15

# format the intervals by truncating the decimal (using int(i.left) ), and into a format "5132 - 94105"
chart_names = [f"{int(i.left)} - {int(i.right)}" for i in bin_counts.index]
# get the count of each salary band/bin e.g. [ 916, 2006,  732,   86,   14]
chart_values = bin_counts.values

      work_year experience_level employment_type                  job_title  \
5          2023               SE              FT          Applied Scientist   
6          2023               SE              FT          Applied Scientist   
9          2023               SE              FT             Data Scientist   
10         2023               SE              FT             Data Scientist   
11         2023               SE              FT               Data Analyst   
...         ...              ...             ...                        ...   
1815       2023               SE              FT  Machine Learning Engineer   
1817       2023               MI              FT             Data Scientist   
1818       2023               MI              FT             Data Scientist   
1819       2023               EN              FT              Data Engineer   
1820       2023               EN              FT              Data Engineer   

      salary salary_currency  salary_in_usd employe

In [59]:
import plotly.express as px
figure = px.pie(names=chart_names, values=chart_values, title='Salary Distribution')
figure.show()

In [62]:
import plotly.graph_objects as go
sorted_indices = sorted(range(len(chart_values)), key=lambda k: chart_values[k])
sorted_chart_names = [chart_names[i] for i in sorted_indices]
sorted_chart_values = [chart_values[i] for i in sorted_indices]
fig = go.Figure(data=[go.Pie(labels=sorted_chart_names, values=sorted_chart_values)])

# Set the chart title
fig.update_layout(title='Salary Bins Distribution')

# Show the chart
fig.show()